In [26]:
from gurobipy import GRB
import gurobipy as gb

In [27]:
import pandas as pd

In [28]:
# Create the model
model = gb.Model("FIBA")

# Read the dataset
df = pd.read_csv('/Users/frieda/Desktop/BasketballPlayers.csv', index_col='Number')

# Create decision variables
x = model.addVars(150, vtype=gb.GRB.BINARY, name="Player")

# Set the objective function
model.setObjective(gb.quicksum(df.iloc[i, 1:8].sum() * x[i] for i in range(150)), gb.GRB.MAXIMIZE)

# Capacity constraint
model.addConstr(gb.quicksum(x[i] for i in range(150)) == 21, "Total Players")

# Position constraints
guard_indices = df[(df['Position'] == 'G') | (df['Position'] == 'G/F')].index.tolist()
model.addConstr(gb.quicksum(x[i] for i in guard_indices) >= 0.3 * 21, "Guard Position")

forward_center_indices = df[(df['Position'] == 'F') | (df['Position'] == 'C') | (df['Position'] == 'F/C')].index.tolist()
if forward_center_indices:
    model.addConstr(gb.quicksum(x[i] for i in forward_center_indices if i < 150) >= 0.4 * 21, "Forward/Center Position")

# Average score constraints
for j in range(7):
    model.addConstr((gb.quicksum(df.iloc[i, j+1] * x[i] for i in range(150)) / 21) >= 2.05, f"Average Score {j}")

# Constraint: if any 20-24, not all 72-78
for i in range(19, 24):
    model.addConstr(x[i] <= 1 - gb.quicksum(x[j] for j in range(71, 78)), f"If any 20-24, not all 72-78")

# Constraint: if any 105-114, at least one 45-49 and at least one 60-69
for i in range(104, 114):
    model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(44, 49)), f"If any 105-114, at least 45-49")
    model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(64, 69)), f"If any 105-114, at least 65-69")

# We need at least one player from every 10 players
for i in range(14):
    model.addConstr(gb.quicksum(x[i*10 + j] for j in range(10)) >= 1, f"At least one player from group {i}")

# Optimize the model
model.optimize()

# Print the objective and decision variables
model.printAttr('X')

# Print the model status
print("Model Status: ", model.status)

# Count and print the number of guards invited
selected = [i for i, var in enumerate(model.getVars()) if var.x != 0]
guards_count = sum(1 for i in selected if df.iloc[i]['Position'] in ['G', 'G/F'])
print(f'Number of G and G/F: {guards_count}')


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 20.2.0 20C69)

CPU model: Intel(R) Core(TM) i5-5257U CPU @ 2.70GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 49 rows, 150 columns and 1649 nonzeros
Model fingerprint: 0xcbe95b5d
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.01s
Presolved: 49 rows, 150 columns, 1310 nonzeros
Variable types: 0 continuous, 150 integer (150 binary)

Root relaxation: objective 3.600000e+02, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     360.0000000  360.00000  0.00%     -    0s

Explored 1 nodes (12 simplex iterations) 

In [31]:
#2h:

def smallest(number):

    x = model.addVars(150, vtype=gb.GRB.BINARY, name="Player")

    model.setObjective(gb.quicksum(((df.iloc[i,1:8]).sum())*x[i] for i in range(150)), gb.GRB.MAXIMIZE)

    model.addConstr(gb.quicksum(x[i] for i in range(150)) == 21, "Total Players")

    guard_indices = df[(df['Position'] == 'G') | (df['Position'] == 'G/F')].index.tolist()
    model.addConstr(gb.quicksum(x[i] for i in guard_indices) >= 0.3 * 21, "Guard Position")

    index_fc = df[(df['Position'] == 'F') | (df['Position'] == 'C') | (df['Position'] == 'F/C')].index.tolist()
    model.addConstr(gb.quicksum(x[i-1] for i in index_fc) >= number * 0.4, "Forward/Center Position")

    forward_center_indices = df[(df['Position'] == 'F') | (df['Position'] == 'C') | (df['Position'] == 'F/C')].index.tolist()
    if forward_center_indices:
        model.addConstr(gb.quicksum(x[i] for i in forward_center_indices if i < 150) >= 0.4 * 21, "Forward/Center Position")

    for j in range(7):
        model.addConstr((gb.quicksum(df.iloc[i, j+1] * x[i] for i in range(150)) / 21) >= 2.05, f"Average Score {j}")

    for i in range(19, 24):
        model.addConstr(x[i] <= 1 - gb.quicksum(x[j] for j in range(71, 78)), f"If any 20-24, not all 72-78")

    for i in range(104, 114): 
        model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(44, 49)), f"If any 105-114, at least 45-49")
        model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(64, 69)), f"If any 105-114, at least 65-69")

    for i in range(14):
        model.addConstr(gb.quicksum(x[i*10 + j] for j in range(10)) >= 1, f"At least one player from group {i}")


    return model

# initial number of smallest number of invitations 
num_of_invite = 21

# Iterate over the range of invitations from the initial value down to 0
for number in range(num_of_invite, 0, -1):
    # Set the number of invitations to the current value
    model = smallest(number)
    
    # Solve the model
    model.optimize()
    
    # Check feasibility status
    if model.status != gb.GRB.Status.OPTIMAL:
        # Infeasible solution found, break the loop
        break



Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 20.2.0 20C69)

CPU model: Intel(R) Core(TM) i5-5257U CPU @ 2.70GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 810 rows, 1650 columns and 18950 nonzeros
Model fingerprint: 0xa4c237dd
Variable types: 0 continuous, 1650 integer (1650 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 681 rows and 1350 columns
Presolve time: 0.15s
Presolved: 129 rows, 300 columns, 2783 nonzeros
Variable types: 0 continuous, 300 integer (300 binary)

Root relaxation: infeasible, 60 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

In [32]:
# Print the result
print("Smallest number of invitations before the model yields an infeasible solution:", num_of_invite+1)

model.computeIIS()
infeasible_constraint = [constr.ConstrName for constr in model.getConstrs() if constr.IISConstr]
print("constraint(s) cannot be satisfied:", infeasible_constraint)

Smallest number of invitations before the model yields an infeasible solution: 22
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 20.2.0 20C69)

CPU model: Intel(R) Core(TM) i5-5257U CPU @ 2.70GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0       810         -         0         0         0           0s
       15        15        15         0         0         0           0s

IIS computed: 15 constraints, 0 bounds
IIS runtime: 0.12 seconds (0.01 work units)
constraint(s) cannot be satisfied: ['Capacity Constraint', 'At least one from group 0', 'At least one from group 1', 'At least one from group 2', 'At least one from group 3', 'At least one from group 4', 'A